terminal: https://python.land/virtual-environments/virtualenv

yingliu@Yings-Air fraud detection in healthcare % source healthfraud/bin/activate

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing


#### Read data 

In [2]:
# data train include provider ID and labels
Train = pd.read_csv("Train-1542865627584.csv")

# include beneficiary KYC details like health conditions,region
Train_Beneficiarydata = pd.read_csv("Train_Beneficiarydata-1542865627584.csv")

# include the claims filed for those patients who are admitted in the hospitals
Train_Inpatientdata=pd.read_csv("Train_Inpatientdata-1542865627584.csv")

# include the claims filed for those patients who visit hospitals and not admitted in it.
Train_Outpatientdata=pd.read_csv("Train_Outpatientdata-1542865627584.csv")


In [3]:
print("Train\n", Train)
print("Train_Beneficiarydata\n", Train_Beneficiarydata)
print("Train_Inpatientdata\n", Train_Inpatientdata)
print("Train_Outpatientdata\n", Train_Outpatientdata)


Train
       Provider PotentialFraud
0     PRV51001             No
1     PRV51003            Yes
2     PRV51004             No
3     PRV51005            Yes
4     PRV51007             No
...        ...            ...
5405  PRV57759             No
5406  PRV57760             No
5407  PRV57761             No
5408  PRV57762             No
5409  PRV57763             No

[5410 rows x 2 columns]
Train_Beneficiarydata
             BeneID         DOB  DOD  Gender  Race RenalDiseaseIndicator  \
0        BENE11001  1943-01-01  NaN       1     1                     0   
1        BENE11002  1936-09-01  NaN       2     1                     0   
2        BENE11003  1936-08-01  NaN       1     1                     0   
3        BENE11004  1922-07-01  NaN       1     1                     0   
4        BENE11005  1935-09-01  NaN       1     1                     0   
...            ...         ...  ...     ...   ...                   ...   
138551  BENE159194  1939-07-01  NaN       1     1           

#### Merging OutPatient ,Inpatient and Beneficiary to get the whole dataset


In [4]:
Train_Allpatientdata = pd.merge(Train_Outpatientdata, Train_Inpatientdata, how='outer')
Train_Allpatientdata = pd.merge(Train_Allpatientdata, Train_Beneficiarydata, on = 'BeneID', how = 'inner')

#### Data preprocessing

In [41]:
# add labels
df = pd.merge(Train_Allpatientdata, Train, on = 'Provider')
df

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmDiagnosisCode_1,...,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,PotentialFraud
0,BENE11002,CLM624349,2009-10-11,2009-10-11,PRV56011,30,PHY326117,NaN,NaN,78943,...,2,2,2,2,2,0,0,30,50,Yes
1,BENE11004,CLM121801,2009-01-06,2009-01-06,PRV56011,40,PHY334319,NaN,NaN,71988,...,1,1,1,1,2,0,0,1810,760,Yes
2,BENE11004,CLM150998,2009-01-22,2009-01-22,PRV56011,200,PHY403831,NaN,NaN,82382,...,1,1,1,1,2,0,0,1810,760,Yes
3,BENE11004,CLM173224,2009-02-03,2009-02-03,PRV56011,20,PHY339887,NaN,NaN,20381,...,1,1,1,1,2,0,0,1810,760,Yes
4,BENE11004,CLM224741,2009-03-03,2009-03-03,PRV56011,40,PHY345721,NaN,NaN,V6546,...,1,1,1,1,2,0,0,1810,760,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558206,BENE57978,CLM37910,2009-02-11,2009-02-16,PRV53511,0,PHY378514,PHY426843,NaN,40491,...,1,1,2,1,2,2200,2136,30,100,No
558207,BENE80245,CLM79653,2009-12-15,2009-12-18,PRV51711,4000,PHY345109,NaN,NaN,2948,...,1,2,2,2,2,4000,1068,0,0,No
558208,BENE85177,CLM77745,2009-11-29,2009-12-07,PRV54503,8000,PHY418343,NaN,NaN,515,...,2,1,2,2,2,8000,1068,0,0,No
558209,BENE105577,CLM53155,2009-05-28,2009-05-31,PRV51239,5000,PHY382916,NaN,NaN,27651,...,1,1,2,2,2,5000,1068,0,0,No


In [42]:
# Drop irrelevant columns
df = df.drop(['BeneID', 'ClaimID'], axis = 1)

In [43]:
#show null values amount
print('null value amount:\n', df.isna().sum())

#show null values percentage
print('null value percentage:\n', df.isnull().mean()*100)

#figure out high percentage columns
nullPercent = df.isnull().mean()*100
filtered_columns = nullPercent[nullPercent > 70].index
print(filtered_columns)



null value amount:
 ClaimStartDt                            0
ClaimEndDt                              0
Provider                                0
InscClaimAmtReimbursed                  0
AttendingPhysician                   1508
OperatingPhysician                 443764
OtherPhysician                     358475
ClmDiagnosisCode_1                  10453
ClmDiagnosisCode_2                 195606
ClmDiagnosisCode_3                 315156
ClmDiagnosisCode_4                 393675
ClmDiagnosisCode_5                 446287
ClmDiagnosisCode_6                 473819
ClmDiagnosisCode_7                 492034
ClmDiagnosisCode_8                 504767
ClmDiagnosisCode_9                 516396
ClmDiagnosisCode_10                553201
ClmProcedureCode_1                 534901
ClmProcedureCode_2                 552721
ClmProcedureCode_3                 557242
ClmProcedureCode_4                 558093
ClmProcedureCode_5                 558202
ClmProcedureCode_6                 558211
DeductibleAmtP

In [44]:
# deal with NA values
# Applying coalesce function to dx columns 
df['ClmDiagnosisCode_1'] = df[[f'ClmDiagnosisCode_{i}' for i in range(1, 11)]].fillna(method='ffill', axis=1)['ClmDiagnosisCode_1']
df['ClmDiagnosisCode_2'] = df[[f'ClmDiagnosisCode_{i}' for i in range(2, 11)]].fillna(method='ffill', axis=1)['ClmDiagnosisCode_2']
df['ClmDiagnosisCode_3'] = df[[f'ClmDiagnosisCode_{i}' for i in range(3, 11)]].fillna(method='ffill', axis=1)['ClmDiagnosisCode_3']


# Applying coalesce function to proc columns 
df['ClmProcedureCode_1'] = df[[f'ClmProcedureCode_{i}' for i in range(1, 7)]].fillna(method='ffill', axis=1)['ClmProcedureCode_1']
df['ClmProcedureCode_2'] = df[[f'ClmProcedureCode_{i}' for i in range(2, 7)]].fillna(method='ffill', axis=1)['ClmProcedureCode_2']



In [45]:
#show null values amount
print('null value amount:\n', df.isna().sum())

#show null values percentage
print('null value percentage:\n', df.isnull().mean()*100)

#figure out high percentage columns
nullPercent = df.isnull().mean()*100
filtered_columns = nullPercent[nullPercent > 70].index
print(filtered_columns)

null value amount:
 ClaimStartDt                            0
ClaimEndDt                              0
Provider                                0
InscClaimAmtReimbursed                  0
AttendingPhysician                   1508
OperatingPhysician                 443764
OtherPhysician                     358475
ClmDiagnosisCode_1                  10453
ClmDiagnosisCode_2                 195606
ClmDiagnosisCode_3                 315156
ClmDiagnosisCode_4                 393675
ClmDiagnosisCode_5                 446287
ClmDiagnosisCode_6                 473819
ClmDiagnosisCode_7                 492034
ClmDiagnosisCode_8                 504767
ClmDiagnosisCode_9                 516396
ClmDiagnosisCode_10                553201
ClmProcedureCode_1                 534901
ClmProcedureCode_2                 552721
ClmProcedureCode_3                 557242
ClmProcedureCode_4                 558093
ClmProcedureCode_5                 558202
ClmProcedureCode_6                 558211
DeductibleAmtP

In [46]:
# remove columns which null value percentage is larger than 70% after processing
df = df.drop(['ClmDiagnosisCode_4', 'ClmDiagnosisCode_5',
       'ClmDiagnosisCode_6', 'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8',
       'ClmDiagnosisCode_9', 'ClmDiagnosisCode_10', 'ClmProcedureCode_1',
       'ClmProcedureCode_2', 'ClmProcedureCode_3', 'ClmProcedureCode_4',
       'ClmProcedureCode_5', 'ClmProcedureCode_6'], axis = 1)

In [47]:
# calculate the number of inpatient days
startdate = pd.to_datetime(df.AdmissionDt)
enddate = pd.to_datetime(df.DischargeDt)
df['Hperiod'] = (enddate - startdate).dt.days

In [48]:
# calculate the period of claim
startdate = pd.to_datetime(df.ClaimStartDt)
enddate = pd.to_datetime(df.ClaimEndDt)
df['Cperiod'] = (enddate - startdate).dt.days

In [50]:
df

,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,...,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,PotentialFraud,Hperiod,Cperiod
0,2009-10-11,2009-10-11,PRV56011,30,PHY326117,NaN,NaN,78943,V5866,V1272,...,2,2,2,0,0,30,50,Yes,NaN,0
1,2009-01-06,2009-01-06,PRV56011,40,PHY334319,NaN,NaN,71988,NaN,NaN,...,1,1,2,0,0,1810,760,Yes,NaN,0
2,2009-01-22,2009-01-22,PRV56011,200,PHY403831,NaN,NaN,82382,30000,72887,...,1,1,2,0,0,1810,760,Yes,NaN,0
3,2009-02-03,2009-02-03,PRV56011,20,PHY339887,NaN,NaN,20381,NaN,NaN,...,1,1,2,0,0,1810,760,Yes,NaN,0
4,2009-03-03,2009-03-03,PRV56011,40,PHY345721,NaN,NaN,V6546,4280,2449,...,1,1,2,0,0,1810,760,Yes,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558206,2009-02-11,2009-02-16,PRV53511,0,PHY378514,PHY426843,NaN,40491,41401,5859,...,2,1,2,2200,2136,30,100,No,5.0,5
558207,2009-12-15,2009-12-18,PRV51711,4000,PHY345109,NaN,NaN,2948,3129,25000,...,2,2,2,4000,1068,0,0,No,3.0,3
558208,2009-11-29,2009-12-07,PRV54503,8000,PHY418343,NaN,NaN,515,5119,4240,...,2,2,2,8000,1068,0,0,No,8.0,8
558209,2009-05-28,2009-05-31,PRV51239,5000,PHY382916,NaN,NaN,27651,25000,5715,...,2,2,2,5000,1068,0,0,No,3.0,3


check for same physician in all three physician columns

In [51]:
def physician_same(row):
    atten_oper = row['AttendingPhysician'] == row['OperatingPhysician']
    oper_other = row['OperatingPhysician'] == row['OtherPhysician']
    atten_other = row['AttendingPhysician'] == row['OtherPhysician']
    
    # atten = oper = other
    if atten_oper == True and oper_other == True:
        return 0
    
    # atten = oper != other
    elif atten_oper == True and oper_other == False:
        return 1
    
    # atten = other != oper
    elif atten_other == True and oper_other == False:
        return 2
    
    # atten != other = oper
    elif atten_other == False and oper_other == True:
        return 3
    
    # atten != other != oper
    else:
        return 4


df['phy_same'] = df.apply(physician_same, axis=1)

encode label and check whether the data is balanced

In [52]:
label_encoder = preprocessing.LabelEncoder()
df['PotentialFraud'] = label_encoder.fit_transform(df['PotentialFraud'])

print(list(label_encoder.classes_))
df['PotentialFraud'].unique()

['No', 'Yes']


array([1, 0])

In [53]:
# 38% positive cases
df['PotentialFraud'].sum()/len(df)

0.3812106891480103

one hot encoding for the categorical features

In [56]:
from sklearn.preprocessing import OneHotEncoder     # One Hot Encoding 

encoder = OneHotEncoder()
one_hot = encoder.fit_transform(df[['Gender']]).toarray()

# df = pd.concat([df, pd.DataFrame(one_hot, columns=encoder.get_feature_names())], axis=1)


# print(df)